# Modeling Carports and Canopies

This journal shows how to model a carport or canopy ~ a fixed structure, usually at a high clearance from the ground, with more than one bifacial solar module in the same inclined-plane to create a "shade" for the cars/people below.

We assume that bifacia_radiacne is already installed in yoru computer. This works for bifacial_radiance v.3 release.

These journal outlines 4 useful uses of bifacial_radiance and some tricks: 

1) Creating the modules in the canopy/carport

2) Adding extra geometry for the pillars/posts supporting the carport/canopy

3) Sampling the rear irradiance and hacking the sensor position to obtain an irradiance map of rear-irradiance. And,

4) Adding a flat surface to simulate a car with a specific reflectivity.

In [1]:
try:
    from bifacial_radiance import *
except ImportError:
    raise RuntimeError('bifacial_radiance is required. download distribution')
    # Simple example system using Radiance.
    
import numpy as np

In [2]:
testfolder = r'C:\Users\sayala\Documents\RadianceScenes\Demo3'

timestamp = 4020 # Noon, June 17th. 
simulationname = 'HotelCarport'

# MakeModule Parameters
moduletype='PrismSolar'
numpanels = 7  # Carport will have 7 modules along the y direction (N-S since we are facing it to the south) .
x = 0.95  
y = 1.95
xgap = 0.15 # Leaving 15 centimeters between modules on x direction
ygap = 0.10 # Leaving 10 centimeters between modules on y direction
zgap = 0 # no gap to torquetube.
sensorsy = 10*numpanels  # this will give 70 sensors per module.

# Other default values:

# TorqueTube Parameters
axisofrotationTorqueTube=False
torqueTube = False
cellLevelModule = False

# SceneDict Parameters
gcr = 0.33   # We are only doing 1 row so this doesn't matter
albedo = 0.28  #'concrete'     # ground albedo
clearance_height = 4.3 # m  
nMods = 15
nRows = 1  # only 1 row

azimuth_ang=180 # Facing south
tilt =20 # tilt. 

# Now let's run the example

demo = RadianceObj(simulationname,path = testfolder)  # Create a RadianceObj 'object'
demo.setGround(albedo) # input albedo number or material name like 'concrete'.  To see options, run this without any input.
epwfile = demo.getEPW(40.0583,-74.4057) # NJ lat/lon 40.0583° N, 74.4057
metdata = demo.readEPW(epwfile) # read in the EPW weather data from above
demo.gendaylit(metdata,4020)  # Use this to simulate only one hour at a time. 
# This allows you to "view" the scene on RVU (see instructions below)
# timestam 4020 : Noon, June 17th.
#demo.genCumSky(demo.epwfile) # Use this for simulate the whole year


# Making module with all the variables
moduleDict=demo.makeModule(name=moduletype,x=x,y=y,numpanels = numpanels, xgap=xgap, ygap=ygap)
# create a scene with all the variables
sceneDict = {'tilt':tilt,'pitch': round(gcr/moduleDict['sceney'],3),'clearance_height':clearance_height,'azimuth':azimuth_ang, 'module_type':moduletype, 'nMods': nMods, 'nRows': nRows}  
scene = demo.makeScene(moduletype=moduletype, sceneDict=sceneDict) #makeScene creates a .rad file with 20 modules per row, 7 rows.
octfile = demo.makeOct(demo.getfilelist())  # makeOct combines all of the ground, sky and object fil|es into a .oct file.


path = C:\Users\sayala\Documents\RadianceScenes\Demo3
Getting weather file: USA_NJ_McGuire.AFB.724096_TMY3.epw
 ... OK!


c:\users\sayala\documents\github\bifacial_radiance\bifacial_radiance\main.py:2409: pvlibDeprecationWarning: The get_sun_rise_set_transit function was deprecated in pvlib 0.6.1 and will be removed in 0.7. Use sun_rise_set_transit_spa instead.
  sunup= pvlib.irradiance.solarposition.get_sun_rise_set_transit(datetimetz, lat, lon) #only for pvlib <0.6.1



Module Name: PrismSolar
REWRITING pre-existing module file. 
Module PrismSolar successfully created
Created HotelCarport.oct


In [ ]:
import numpy as np

xright= x*8
xleft=  -xright

#centerhubheight = (1.9*3+1.9/2)*np.sin(tilt*np.pi/180)
z6th =  (y*6)*np.sin(tilt*np.pi/180)+clearance_height
y6th=  -(y*numpanels/2)*np.cos(tilt*np.pi/180) + (y*6)*np.cos(tilt*np.pi/180)
z2nd = (y*np.sin(tilt*np.pi/180))+clearance_height
y2nd = -(y*numpanels/2)*np.cos(tilt*np.pi/180) + (y)*np.cos(tilt*np.pi/180)
             
name='Post1'
text='! genbox black cuteBox 0.5 0.5 {} | xform -t {} {} 0'.format(z2nd, xleft, y2nd)
print (text)
customObject = demo.makeCustomObject(name,text)
demo.appendtoScene(radfile=scene.radfiles, customObject=customObject, text="!xform -rz 0")

print ("Here")
name='Post2'
text='! genbox black cuteBox 0.5 0.5 {} | xform -t {} {} 0'.format(z2nd, xright, y2nd)
customObject = demo.makeCustomObject(name,text)
demo.appendtoScene(scene.radfiles, customObject, '!xform -rz 0')

name='Post3'
text='! genbox black cuteBox 0.5 0.5 {} | xform -t {} {} 0'.format(z6th, xright, y6th)
customObject = demo.makeCustomObject(name,text)
demo.appendtoScene(scene.radfiles, customObject, '!xform -rz 0')

name='Post4'
text='! genbox black cuteBox 0.5 0.5 {} | xform -t {} {} 0'.format(z6th, xleft, y6th)
customObject = demo.makeCustomObject(name,text)
demo.appendtoScene(scene.radfiles, customObject, '!xform -rz 0')


octfile = demo.makeOct(demo.getfilelist())  # makeOct combines all of the ground, sky and object files into a .oct file.


In [ ]:
testfolder = r'C:\Users\sayala\Documents\RadianceScenes\Demo3'

timestamp = 4020 # Noon, June 17th. 
simulationname = 'ParkingCarport'

# MakeModule Parameters
moduletype='module_NJ'
numpanels = 8
x = 0.95  
y = 1.9 
xgap = 0.15 # 10 centimeters.
ygap = 0.10 # you can give ygap now
zgap = 0
sensorsy = 12*numpanels  # this will give 70 sensors per module.

# TorqueTube Parameters
axisofrotationTorqueTube=False
torqueTube = False
cellLevelModule = False

# SceneDict Parameters
gcr = 0.33   # We are only doing 1 row so this doesn't matter
albedo = 0.28  #'concrete'     # ground albedo
clearance_height = 4.3 # m  
nMods = 45
nRows = 1  # only 1 row

azimuth_ang=90 # Facing south
tilt =20 # tilt. 

# Now let's run the example

demo = RadianceObj(simulationname,path = testfolder)  # Create a RadianceObj 'object'
demo.setGround(albedo) # input albedo number or material name like 'concrete'.  To see options, run this without any input.
epwfile = demo.getEPW(40.0583,-74.4057) # NJ lat/lon 40.0583° N, 74.4057
metdata = demo.readEPW(epwfile) # read in the EPW weather data from above
demo.gendaylit(metdata,4020)  # Use this to simulate only one hour at a time. 
# This allows you to "view" the scene on RVU (see instructions below)
# timestam 4020 : Noon, June 17th.
#demo.genCumSky(demo.epwfile) # Use this for simulate the whole year


# Making module with all the variables
moduleDict=demo.makeModule(name=moduletype,x=x,y=y, numpanels = numpanels, xgap=xgap, ygap=ygap, cellLevelModule=True)
# create a scene with all the variables
sceneDict = {'tilt':tilt,'pitch': round(gcr/moduleDict['sceney'],3),'height':clearance_height,'azimuth':azimuth_ang, 'module_type':moduletype, 'nMods': nMods, 'nRows': nRows}  
scene = demo.makeScene(moduletype=moduletype, sceneDict=sceneDict) #makeScene creates a .rad file with 20 modules per row, 7 rows.
octfile = demo.makeOct(demo.getfilelist())  # makeOct combines all of the ground, sky and object fil|es into a .oct file.


### Adding the pillars/posts

We will add 2 posts at roughly the back corners of the structure. 

In [ ]:
import numpy as np

xright= x*nMods/2
xleft=  -xrigt

# Calculating the height and location of the posts
#centerhubheight = (1.9*3+1.9/2)*np.sin(tilt*np.pi/180)
z6th =  (y*6)*np.sin(tilt*np.pi/180)+clearance_height
y6th=  -(y*numpanels/2)*np.cos(tilt*np.pi/180) + (y*6)*np.cos(tilt*np.pi/180)
             
name='Post3'
text='! genbox black cuteBox 0.5 0.5 {} | xform -t {} {} 0'.format(z6th, y6th, xright)
customObject = demo.makeCustomObject(name,text)
demo.appendtoScene(scene.radfiles, customObject, '!xform -rz 0')

name='Post4'
text='! genbox black cuteBox 0.5 0.5 {} | xform -t {} {} 0'.format(z6th, y6th, xleft)
customObject = demo.makeCustomObject(name,text)
demo.appendtoScene(scene.radfiles, customObject, '!xform -rz 0')

octfile = demo.makeOct(demo.getfilelist())  # makeOct combines all of the ground, sky and object files into a .oct file.


In [ ]:
analysis = AnalysisObj(octfile, demo.name)  # return an analysis object including the scan dimensions for back irradiance
modWanted = 1
rowWanted = 1
frontscan, backscan = analysis.moduleAnalysis(clearanceheight=sceneDict['height'], azimuth=sceneDict['azimuth'], 
                                              tilt=sceneDict['tilt'], 
                                              pitch=sceneDict['pitch'], nMods=sceneDict['nMods'], nRows= sceneDict['nRows'], 
                                              sceney=moduleDict['sceney'], scenex=moduleDict['scenex'], offset=moduleDict['moduleoffset'],
                                              modWanted=modWanted, rowWanted=rowWanted, sensorsy=sensorsy)

analysis.analysis(octfile, simulationname+"Mod1", frontscan, backscan)  # compare the back vs front irradiance  
print('Annual bifacial ratio average:  %0.3f' %( sum(analysis.Wm2Back) / sum(analysis.Wm2Front) ) )


modWanted = 2
rowWanted = 1
frontscan, backscan = analysis.moduleAnalysis(clearanceheight=sceneDict['height'], azimuth=sceneDict['azimuth'], 
                                              tilt=sceneDict['tilt'], 
                                              pitch=sceneDict['pitch'], nMods=sceneDict['nMods'], nRows= sceneDict['nRows'], 
                                              sceney=moduleDict['sceney'], scenex=moduleDict['scenex'], offset=moduleDict['moduleoffset'],
                                              modWanted=modWanted, rowWanted=rowWanted, sensorsy=sensorsy)

analysis.analysis(octfile, simulationname+"Mod2", frontscan, backscan)  # compare the back vs front irradiance  
print('Annual bifacial ratio average:  %0.3f' %( sum(analysis.Wm2Back) / sum(analysis.Wm2Front) ) )


modWanted = 3
rowWanted = 1
frontscan, backscan = analysis.moduleAnalysis(clearanceheight=sceneDict['height'], azimuth=sceneDict['azimuth'], 
                                              tilt=sceneDict['tilt'], 
                                              pitch=sceneDict['pitch'], nMods=sceneDict['nMods'], nRows= sceneDict['nRows'], 
                                              sceney=moduleDict['sceney'], scenex=moduleDict['scenex'], offset=moduleDict['moduleoffset'],
                                              modWanted=modWanted, rowWanted=rowWanted, sensorsy=sensorsy)

        
analysis.analysis(octfile, simulationname+"Mod3", frontscan, backscan)  # compare the back vs front irradiance  
print('Annual bifacial ratio average:  %0.3f' %( sum(analysis.Wm2Back) / sum(analysis.Wm2Front) ) )

In [ ]:
# HACK Frontscan and Backscan
frontscan['xstart']=-1.2

        
analysis.analysis(octfile, simulationname+"Mod3_point2", frontscan, backscan)  # compare the back vs front irradiance  
print('Annual bifacial ratio average:  %0.3f' %( sum(analysis.Wm2Back) / sum(analysis.Wm2Front) ) )
